# 1: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

In [1]:
import pandas as pd
import json
import sqlite3


##### Loading json data to pandas df

In [2]:
file_path_users = "/Users/agupota/Documents/Apoorva/Fetch/users.json"
file_path_receipts = "/Users/agupota/Documents/Apoorva/Fetch/receipts.json"
file_path_brands = "/Users/agupota/Documents/Apoorva/Fetch/brands.json"

## 1 Load users.json file

In [3]:
with open(file_path_users, "r", encoding="utf-8") as file:
            users_json = [json.loads(line) for line in file]
users = pd.json_normalize(users_json)
users.columns = users.columns.str.replace(r'[\._$]', '', regex=True)
users = users.rename(columns = {'idoid': 'userId', 'createdDatedate': 'createdDate', 
                                      'lastLogindate': 'lastLogin'})
users

,active,role,signUpSource,state,userId,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
...,...,...,...,...,...,...,...
490,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
491,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
492,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
493,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12


In [4]:
print("Total rows:",users['userId'].count())
print("Unique userids",users['userId'].nunique())
print("Users by role:",users.groupby('role')['userId'].nunique().reset_index())
print("Users by active:",users.groupby('active')['userId'].nunique().reset_index())

Total rows: 495
Unique userids 212
Users by role:           role  userId
0     consumer     204
1  fetch-staff       8
Users by active:    active  userId
0   False       1
1    True     211


### **My Findings:**
1. Total rows are 495, but unique `userIds` are 212. There must be duplicates, will remove dupliacted data
2. Column active has data - True and False which means its Boolean

In [5]:
users = users.drop_duplicates()
print("Total rows:",users['userId'].count())
print("Unique userids",users['userId'].nunique())
print("Users by role:",users.groupby('role')['userId'].nunique().reset_index())
print("Users by active:",users.groupby('active')['userId'].nunique().reset_index())
print("Users by signUpSource:",users.groupby('signUpSource')['userId'].nunique().reset_index())

Total rows: 212
Unique userids 212
Users by role:           role  userId
0     consumer     204
1  fetch-staff       8
Users by active:    active  userId
0   False       1
1    True     211
Users by signUpSource:   signUpSource  userId
0        Email     204
1       Google       3


In [6]:
# convert timestamps to datetime format
users['createdDate'] = pd.to_datetime(users['createdDate'] // 1000, unit='s')
users['lastLogin'] = pd.to_datetime(users['lastLogin'] // 1000, unit='s')

/var/folders/gb/mmz5qdk97xb0z6x26pvg9jlw0000gp/T/ipykernel_72459/1495051154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['createdDate'] = pd.to_datetime(users['createdDate'] // 1000, unit='s')
/var/folders/gb/mmz5qdk97xb0z6x26pvg9jlw0000gp/T/ipykernel_72459/1495051154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['lastLogin'] = pd.to_datetime(users['lastLogin'] // 1000, unit='s')


In [7]:
users.describe()

,createdDate,lastLogin
count,212,172
mean,2020-12-09 11:04:29.957547264,2021-01-15 21:40:01.773255680
min,2014-12-19 14:21:22,2018-05-07 17:23:40
25%,2021-01-08 17:26:41.500000,2021-01-08 20:39:16.249999872
50%,2021-01-19 20:42:49,2021-01-20 17:36:50.500000
75%,2021-01-27 23:11:30.750000128,2021-01-29 18:56:54.750000128
max,2021-02-12 14:11:06,2021-03-05 16:52:23


### 2 Load brands.json file

In [8]:
with open(file_path_brands, "r", encoding="utf-8") as file:
            brands_json = [json.loads(line) for line in file]
brands = pd.json_normalize(brands_json)
brands.columns = brands.columns.str.replace(r'[\._$]', '', regex = True)
brands = brands.rename(columns = {'idoid': 'brandId', 'cpgidoid': 'cpgOid', 'cpgref': 'cpgRef', 
                                  'name' : 'brandName'})
brands

,barcode,category,categoryCode,brandName,topBrand,brandId,cpgOid,cpgRef,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [9]:
print("Total rows:",brands['brandId'].count())
print("Unique brandis",brands['brandId'].nunique())
print("Unique barcodes",brands['barcode'].nunique())
print("Unique cpgids",brands['cpgOid'].nunique())


Total rows: 1167
Unique brandis 1167
Unique barcodes 1160
Unique cpgids 196


In [10]:
print("category code is null but category has data",brands[~(brands['category'].isnull()) &
                                                           (brands['categoryCode'].isnull())]['brandId'].count())
print("category is null but category code has data",brands[(brands['category'].isnull()) & 
                                                          ~(brands['categoryCode'].isnull())]['brandId'].count())
brands[['category','categoryCode']].value_counts()

category code is null but category has data 495
category is null but category code has data 0


category                     categoryCode                 
Baking                       BAKING                           359
Candy & Sweets               CANDY_AND_SWEETS                  71
Beer Wine Spirits            BEER_WINE_SPIRITS                 31
Health & Wellness            HEALTHY_AND_WELLNESS              14
Grocery                      GROCERY                           11
Baby                         BABY                               7
Cleaning & Home Improvement  CLEANING_AND_HOME_IMPROVEMENT      6
Bread & Bakery               BREAD_AND_BAKERY                   5
Dairy & Refrigerated         DAIRY_AND_REFRIGERATED             5
Personal Care                PERSONAL_CARE                      4
Beverages                    BEVERAGES                          1
Frozen                       FROZEN                             1
Magazines                    MAGAZINES                          1
Outdoor                      OUTDOOR                            1
Name: count, dtyp

### **My Findings:**
1. Total rows are 1167 and unique brandIds are 1167. There must be no duplicates at brandId level.
2. I noticed that catgeory and categoryCode has same data. categoryCode has data in capital letters, use of AND in place of &. And then category column has more data than categoryCode.

In [11]:
# Changed the format of catgeory and categorycode to be same
brands['category'] = brands['category'].str.replace('_', ' ').str.upper()
brands['categoryCode'] = brands['categoryCode'].str.replace('_', ' ').str.upper()
brands['categoryCode'] = brands['categoryCode'].str.replace(' AND ', ' & ').str.upper()
brands[brands['category'] != brands['categoryCode']][['category','categoryCode']].value_counts()

category           categoryCode      
HEALTH & WELLNESS  HEALTHY & WELLNESS    14
Name: count, dtype: int64

### **My Findings:**
1. Category column and categoryCode column has same data and categoryCode is null for 495 rows but category has data so we can exclude categoryCode from our table in order to reduce the redundant data and normalize the table

In [12]:
brands = brands.drop(columns=['categoryCode'],axis=1)

In [13]:
print(brands.groupby('topBrand')['brandId'].nunique().reset_index())
print(brands.groupby('cpgRef')['brandId'].nunique().reset_index())

   topBrand  brandId
0     False      524
1      True       31
  cpgRef  brandId
0   Cogs     1020
1   Cpgs      147


In [14]:
# Checking if brandName and brandCode have same data

### making brandName and brandCode in same format
brands['brandName'] = brands['brandName'].str.upper()
brands['brandCode'] = brands['brandCode'].str.upper()
brands['brandCode'] = brands['brandCode'].str.replace('CODE', '', regex=False).str.strip()
brands['brandName'] = brands['brandName'].str.replace(r"[^\w\s]", "", regex=True)
brands['brandCode'] = brands['brandCode'].str.replace(r"[^\w\s]", "", regex=True)
brands[(brands['brandName']!=brands['brandCode']) & ~(brands['brandCode'].isnull()) & (brands['brandCode']!="")]

,barcode,category,brandName,topBrand,brandId,cpgOid,cpgRef,brandCode
13,511111205012,MAGAZINES,ENTERTAINMENT WEEKLY,NaN,5d6413156d5f3b23d1bc790a,5d5d4fd16d5f3b23d1bc7905,Cogs,511111205012
15,511111202233,BEER WINE SPIRITS,MOLSON CANADIAN,False,57e5820ce4b0ac389136a311,5332f709e4b03c9a25efd0f1,Cpgs,MOLSON
20,511111305125,BABY,CHRIS IMAGE TEST,NaN,5c4699f387ff3577e203ea29,55b62995e4b0d8e685c14213,Cogs,CHRISIMAGE
21,511111005650,HEALTH & WELLNESS,ALKASELTZER,NaN,5da6071ea60b87376833e34d,5d9b4f591dda2c6225a284aa,Cogs,ALKA SELTZER
22,511111802129,CONDIMENTS & SAUCES,JACK DANIELS,False,57ebc011e4b0ac389136a335,559c2234e4b06aca36af13c6,Cogs,JACK DANIELS BARBECUE
...,...,...,...,...,...,...,...,...
1150,511111005377,MAGAZINES,MODERN FARMHOUSE MAGAZINE,NaN,5d66da306d5f3b6188d4f04c,5d5d4fd16d5f3b23d1bc7905,Cogs,511111005377
1151,511111905479,MAGAZINES,WEIGHT WATCHERS SPECIAL EDITION MAGAZINE,NaN,5d66dfe6a3a018093ab3472c,5d5d4fd16d5f3b23d1bc7905,Cogs,511111905479
1156,511111617853,GROCERY,DITALIANO,False,5fb82fe3be37ce522e165cea,5fb6b608be37ce522e165cb9,Cogs,D ITALIANO
1158,511111716648,BAKING,TEST BRAND 1600291349042,NaN,5f628215be37ce78e6e2efab,5f628214be37ce78e6e2efaa,Cogs,TEST BRAND 1600291349043


### **My Findings:**
1. brandName and brandCode have different data for 214 rows, so I decide to keep both columns
2. In order to structure the relational data model we have to create 2 tables - one table brands and another table with cpg

In [15]:
brands.groupby('cpgOid')['cpgRef'].nunique() \
    .rename('cnt') \
    .reset_index() \
    .groupby('cnt')['cpgOid'] \
    .count()

cnt
1    188
2      8
Name: cpgOid, dtype: int64

### **My Findings:**
1. 8 cpgOids have multiple value. Lets assume that it is data discrepancy and replace with single cogs value for them 

In [16]:
cpg_ref_counts = brands.groupby('cpgOid')['cpgRef'].nunique()
multiple_cpg_ref = cpg_ref_counts[cpg_ref_counts > 1].index

brands.loc[brands['cpgOid'].isin(multiple_cpg_ref), 'cpgRef'] = 'cogs'
cpg = brands[['cpgOid', 'cpgRef']].drop_duplicates()

In [17]:
brands.describe()

,barcode,category,brandName,topBrand,brandId,cpgOid,cpgRef,brandCode
count,1167,1012,1167,555,1167,1167,1167,933
unique,1160,23,1152,2,1167,196,3,892
top,511111305125,BAKING,DIGIORNO CHEESE,False,601ac115be37ce2ead437551,559c2234e4b06aca36af13c6,Cogs,
freq,2,369,2,524,1,98,693,35


## 3 Load receipts.json file

In [18]:
with open(file_path_receipts, "r", encoding="utf-8") as file:
            receipts_json = [json.loads(line) for line in file]
receipts = pd.json_normalize(receipts_json)
receipts.columns = receipts.columns.str.replace(r'[\._$]', '', regex = True)
receipts = receipts.rename(columns = {'idoid': 'receiptId', 'createDatedate': 'createDate', 
                                      'dateScanneddate': 'dateScanned', 'finishedDatedate': 'finishedDate', 
                                      'modifyDatedate': 'modifyDate', 'purchaseDatedate':'purchaseDate',
                                      'pointsAwardedDatedate':'pointsAwardedDate'})
receipts

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,receiptId,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,1614594147000,1614594147000,NaN,1614594148000,NaN,1.597622e+12
1115,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN
1116,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN
1117,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603ce7100a7217c72c000405,1614604048000,1614604048000,NaN,1614604049000,NaN,1.597622e+12


### **My Findings:**
1. rewardsReceiptItemList has another list of nested data that contains items that were purchased on receipt. Lets save that data in another table in order to create structured data model

In [19]:
item_receipts = receipts[['receiptId', 'rewardsReceiptItemList']].explode('rewardsReceiptItemList')
item_receipts = item_receipts.dropna(subset=['rewardsReceiptItemList'])
items_df = pd.json_normalize(item_receipts['rewardsReceiptItemList'])
final_item_receipts = item_receipts[[ 'receiptId']].reset_index(drop=True).join(items_df)
final_item_receipts.columns = final_item_receipts.columns.str.replace(r'[\._$]', '', regex=True)
final_item_receipts

,receiptId,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,603cc2bc0a720fde100003e9,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,603cc0630a720fde100003e6,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,603cc0630a720fde100003e6,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [20]:
print(receipts['userId'].count())
print(receipts['userId'].nunique())
print(receipts['receiptId'].count())
print(receipts['receiptId'].nunique())
print(receipts['rewardsReceiptStatus'].value_counts())

1119
258
1119
1119
rewardsReceiptStatus
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64


### **My Findings:**
1. Total no of rows are 1119 and unique receipt id count is 1119. There are no duplicates at receiptId level
2. Since rewardsReceiptItemList is stored in final_item_receipts table, we can delete from receipts to reduce the data redundancy
3. Also, notice that here unique users are 258 but in users table unique users are 212

In [21]:
receipts = receipts.drop(columns = ['rewardsReceiptItemList'],axis=1)

In [22]:
# convert epoch time columns to datetime format
time_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in time_columns:
    receipts[col] = pd.to_datetime(receipts[col] // 1000, unit='s', errors='coerce')

In [23]:
final_item_receipts = final_item_receipts.drop_duplicates()
print(final_item_receipts['receiptId'].count())
print(final_item_receipts['receiptId'].nunique())
print(final_item_receipts['partnerItemId'].count())
print(final_item_receipts['partnerItemId'].nunique())
print(final_item_receipts[['receiptId','partnerItemId']].drop_duplicates()['receiptId'].count())

6941
679
6941
916
6941


In [24]:
###### check how brandCode from barnds is related with brandCode in final_item_receipts table
print("no of rows with brandcode and brandcode match",final_item_receipts[(final_item_receipts['brandCode'].
                                                            isin(brands['brandCode']))]['receiptId'].count())
print("no of rows with barcode and brandcode match",final_item_receipts[(final_item_receipts['barcode'].
                                                            isin(brands['brandCode']))]['receiptId'].count())
print("no of rows with (brandcode or barcode) and brandcode match",final_item_receipts[(final_item_receipts['brandCode'].
                                                                isin(brands['brandCode'])) | \
                 (final_item_receipts['barcode'].isin(brands['brandCode']))]['receiptId'].count())

no of rows with brandcode and brandcode match 4947
no of rows with barcode and brandcode match 3851
no of rows with (brandcode or barcode) and brandcode match 5788


### **My Findings:**
1. Level of final_item_receipts table is receiptId, partnerItemId. The combination of both table should be the primary key
2. Since there are more matches with brandCode in final_item_receipts table with brandCode in brand table. I will make brandCode as foreign key. But while writing queries I will use both columns to join back to brand data

In [25]:
receipts.select_dtypes(include=['number']).describe()

,bonusPointsEarned,purchasedItemCount
count,544.000000,635.00000
mean,238.893382,14.75748
std,299.091731,61.13424
min,5.000000,0.00000
25%,5.000000,1.00000
50%,45.000000,2.00000
75%,500.000000,5.00000
max,750.000000,689.00000


In [26]:
final_item_receipts.select_dtypes(include=['number']).describe()

,quantityPurchased,userFlaggedQuantity,originalMetaBriteQuantityPurchased
count,6767.000000,299.000000,15.000000
mean,1.386139,1.872910,1.200000
std,1.204363,1.314823,0.414039
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000
max,17.000000,5.000000,2.000000


## Insert data in sql tables

In [27]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

##### users 
cursor.execute('''
    CREATE TABLE users (
        userId VARCHAR(255) PRIMARY KEY,
        role VARCHAR(255),
        active BOOLEAN,
        signUpSource VARCHAR(10),
        state VARCHAR(10),
        createdDate DATETIME,
        lastLogin DATETIME
    );
''')

##### cpg
cursor.execute('''
    CREATE TABLE cpg (
    cpgOid VARCHAR(255) PRIMARY KEY,
    cpgRef VARCHAR(20)
    );
''')
    
##### brands
cursor.execute('''
    CREATE TABLE brands (
    brandId VARCHAR(255) PRIMARY KEY,
    cpgOid VARCHAR(255),
    brandName VARCHAR(255),
    brandCode VARCHAR(255),
    topBrand BOOLEAN,
    category VARCHAR(255),
    barcode VARCHAR(255),
    FOREIGN KEY (cpgOid) REFERENCES cpg(cpgOid)
    );
''')

##### receipts
cursor.execute('''
    CREATE TABLE IF NOT EXISTS receipts (
    receiptId VARCHAR(255) PRIMARY KEY,
    userId VARCHAR(255),
    rewardsReceiptStatus VARCHAR(20),
    bonusPointsEarned FLOAT,
    bonusPointsEarnedReason VARCHAR(255),
    pointsEarned FLOAT,
    purchasedItemCount INTEGER,
    totalSpent FLOAT,
    createDate DATETIME,
    dateScanned DATETIME,
    finishedDate DATETIME,
    modifyDate DATETIME,
    pointsAwardedDate DATETIME,
    purchaseDate DATETIME,
    FOREIGN KEY (userId) REFERENCES users(userId)
    );
''')

#### receiptItems
cursor.execute('''
    CREATE TABLE IF NOT EXISTS receiptItems (
    receiptId VARCHAR(255),
    partnerItemId VARCHAR(255),
    barcode VARCHAR(255),
    description TEXT,
    finalPrice TEXT,
    itemPrice TEXT,
    needsFetchReview TEXT,
    preventTargetGapPoints TEXT,
    quantityPurchased FLOAT,
    userFlaggedBarcode TEXT,
    userFlaggedNewItem TEXT,
    userFlaggedPrice TEXT,
    userFlaggedQuantity FLOAT,
    needsFetchReviewReason TEXT,
    pointsNotAwardedReason TEXT,
    pointsPayerId TEXT,
    rewardsGroup TEXT,
    rewardsProductPartnerId TEXT,
    userFlaggedDescription TEXT,
    originalMetaBriteBarcode TEXT,
    originalMetaBriteDescription TEXT,
    brandCode TEXT,
    competitorRewardsGroup TEXT,
    discountedItemPrice TEXT,
    originalReceiptItemText TEXT,
    itemNumber TEXT,
    originalMetaBriteQuantityPurchased FLOAT,
    pointsEarned TEXT,
    targetPrice TEXT,
    competitiveProduct TEXT,
    originalFinalPrice TEXT,
    originalMetaBriteItemPrice TEXT,
    deleted TEXT,
    priceAfterCoupon TEXT,
    metabriteCampaignId TEXT,
    PRIMARY KEY (receiptId, partnerItemId),
    FOREIGN KEY (receiptId) REFERENCES receipts(receiptId),
    FOREIGN KEY (brandCode) REFERENCES brands(brandCode)
    );
''')


In [28]:
def insert_and_check(df, table_name, conn):
    initial_count = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", conn).iloc[0, 0]
    df.to_sql(table_name, conn, if_exists='append', index=False)
    final_count = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", conn).iloc[0, 0]
    inserted_rows = final_count - initial_count
    not_inserted_rows = len(df) - inserted_rows
    
    if not_inserted_rows > 0:
        print(f"{not_inserted_rows} rows were not inserted into {table_name}.")
    else:
        print(f"All rows were successfully inserted into {table_name}.")

In [29]:
insert_and_check(users, 'users', conn)
insert_and_check(cpg, 'cpg', conn)
brands = brands[['brandId', 'cpgOid', 'brandName', 'brandCode', 'topBrand', 'category', 'barcode']]
insert_and_check(brands, 'brands', conn)
insert_and_check(receipts, 'receipts', conn)
insert_and_check(final_item_receipts, 'receiptItems', conn)

All rows were successfully inserted into users.
All rows were successfully inserted into cpg.
All rows were successfully inserted into brands.
All rows were successfully inserted into receipts.
All rows were successfully inserted into receiptItems.


# 2: Write queries that directly answer predetermined questions from a business stakeholder

## 1. What are the top 5 brands by receipts scanned for most recent month?

In [30]:
### check how many receipts are there in each month and out of them how many are able to map
query = """
    with total_receipts as 
    (select strftime('%Y-%m', dateScanned) AS scanneddate, count(distinct receiptId) as tot_rcpt
        FROM receipts
        group by 1
        order by 1 desc
    ),
    
    mapped_brand as (
    select scanneddate, count(receiptId) as  mapped_recpt
    from 
        (select rcpt.scanneddate,rcpt.receiptId, rcpt.brandCode, br.brandId, br.brandName
        from
            (select strftime('%Y-%m', dateScanned) AS scanneddate, r.receiptId, 
            case when brandCode is not null then brandCode else barcode end as brandCode
            from receipts as r
            inner join receiptItems as ri on ri.receiptId = r.receiptId
            ) as rcpt
        inner join brands as br on rcpt.brandCode = br.brandCode
        )
    group by  scanneddate
    )

select a.*, b.mapped_recpt
from total_receipts as a
left join mapped_brand as b on a.scanneddate = b.scanneddate
    
"""
scanned_df_count = pd.read_sql(query, conn)
scanned_df_count

,scanneddate,tot_rcpt,mapped_recpt
0,2021-03,30,NaN
1,2021-02,444,4.0
2,2021-01,637,608.0
3,2020-11,6,NaN
4,2020-10,2,NaN


### **My Findings:**
1. Only 30 receipts were there in month of march, 2021. It seems that we dont have full data of march. 
2. only Jan, 2021 has brand mapped data. In order to get the correct data, we assume Jan, 2021 is most recent data

In [31]:
query = """
with recent_month_recpt AS (
    select distinct r.receiptId, strftime('%Y-%m', r.dateScanned) as month_scanned
    from receipts r
    where strftime('%Y-%m', r.dateScanned) = '2021-01'
),

RankedBrands AS (
    select 
        r.month_scanned,
        br.brandName,
        count(ri.receiptId) AS receipt_count,
        row_number() over (order by count(ri.receiptId) DESC) AS rank
    from 
        receiptItems ri
    inner join recent_month_recpt r on ri.receiptId = r.receiptId
    inner join brands br on case when ri.brandCode is not null then ri.brandCode else ri.barcode end = br.brandCode
    group by r.month_scanned,br.brandName
)

select month_scanned,brandName, receipt_count, rank
from RankedBrands
where rank <= 5
order by rank;
"""
top_brands = pd.read_sql(query, conn)
top_brands

,month_scanned,brandName,receipt_count,rank
0,2021-01,PEPSI,93,1
1,2021-01,KLEENEX,88,2
2,2021-01,KNORR,79,3
3,2021-01,DORITOS,77,4
4,2021-01,KRAFT,60,5


## 2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [32]:
##### Assumption: Feb, 2021 is recent month and Jan 2021 is previous month in order to get some results

query = """
with recent_month_recpt AS (
    select distinct r.receiptId, strftime('%Y-%m', r.dateScanned) AS recentmonth_scanned
    from receipts r
    where strftime('%Y-%m', r.dateScanned) = '2021-02'
),

previous_month_recpt AS (
    select distinct r.receiptId, strftime('%Y-%m', r.dateScanned) AS prevmonth_scanned
    from receipts r
    where strftime('%Y-%m', r.dateScanned) = '2021-01'
),

RankedBrands_recent AS (
    select 
        r.recentmonth_scanned,
        br.brandName,
        count(ri.receiptId) AS receipt_count,
        row_number() over (order by count(ri.receiptId) desc) AS rank
    from 
        receiptItems ri
    inner join recent_month_recpt r on ri.receiptId = r.receiptId
    inner join brands br on case when ri.brandCode is not null then ri.brandCode else ri.barcode end = br.brandCode
    group by r.recentmonth_scanned,br.brandName
),

RankedBrands_prev AS (
    select 
        r.prevmonth_scanned,
        br.brandName,
        count(ri.receiptId) AS receipt_count,
        row_number() over (order by count(ri.receiptId) DESC) AS rank
    from 
        receiptItems ri
    inner join previous_month_recpt r on ri.receiptId = r.receiptId
    inner join brands br on case when ri.brandCode is not null then ri.brandCode else ri.barcode end = br.brandCode
    group by r.prevmonth_scanned,br.brandName
)

select a.*,  b.prevmonth_scanned, b.rank as premonth_rank
from RankedBrands_recent as a
left join RankedBrands_prev as b on a.brandName = b.brandName
where a.rank <= 5
order by a.rank;

"""
top_brands_rank_perf = pd.read_sql(query, conn)
top_brands_rank_perf

,recentmonth_scanned,brandName,receipt_count,rank,prevmonth_scanned,premonth_rank
0,2021-02,7 UP,3,1,2021-01,7.0
1,2021-02,VIVA,1,2,None,NaN


## 3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [33]:
######## checking avg spend by each status
query = """
select 
    r.rewardsReceiptStatus,
    sum(totalSpent) AS avg_spend
from receipts r
group by 1;

"""
status_check = pd.read_sql(query, conn)
status_check

,rewardsReceiptStatus,avg_spend
0,FINISHED,41882.53
1,FLAGGED,8300.78
2,PENDING,1373.59
3,REJECTED,1656.15
4,SUBMITTED,NaN


### **My Findings:**
1. Since there is no Accepted status and by looking data, I assume Finished is accepted status

In [34]:
query = """
select 
   case when upper(r.rewardsReceiptStatus) in ('FINISHED') then "Accepted" 
    when upper(r.rewardsReceiptStatus) = 'REJECTED' then 'Rejected' end as rewardsReceiptStatus,
    round(AVG(totalSpent),2) AS avg_spend
from receipts r
where upper(r.rewardsReceiptStatus) IN ('FINISHED','REJECTED')
group by rewardsReceiptStatus;

"""
avgspend_perf = pd.read_sql(query, conn)
avgspend_perf

,rewardsReceiptStatus,avg_spend
0,Accepted,80.85
1,Rejected,23.33


### **My Findings:**
1. Avg spend of Accepted is greater than avg spend of Rejected by 71%

## 4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [35]:
query = """
select 
   case when upper(r.rewardsReceiptStatus) in ('FINISHED') then "Accepted" 
    when upper(r.rewardsReceiptStatus) = 'REJECTED' then 'Rejected' end as rewardsReceiptStatus,
    sum(purchasedItemCount) AS totalitems
from receipts r
where upper(r.rewardsReceiptStatus) IN ('FINISHED','REJECTED')
group by rewardsReceiptStatus;
"""
itemscnt_perf = pd.read_sql(query, conn)
itemscnt_perf

,rewardsReceiptStatus,totalitems
0,Accepted,8184
1,Rejected,173


### **My Findings:**
1. Total number of items purchased from receipts of Accepted is greater than Rejected 

## 5. Which brand has the most spend among users who were created within the past 6 months?

In [36]:
query = """
select b.brandName, sum(r.totalSpent) as totalspent
FROM receipts r
inner join receiptItems ri on r.receiptId = ri.receiptId
inner join brands b on case when ri.brandCode is not null then ri.brandCode else ri.barcode end = b.brandCode
inner join users u on r.userId = u.userId
where date(u.createdDate) between 
      (select date(max(createdDate), '-6 months') from users) 
      and 
      (select date(max(createdDate)) from users)
group by  b.brandName
order by sum(r.totalSpent) desc
limit 1
"""
mostspendBrand_perf = pd.read_sql(query, conn)
mostspendBrand_perf

,brandName,totalspent
0,PEPSI,78870.86


### **My Findings:**
1. Brand - PEPSI  has most spend amoung users who created userid within past 6 months

## 6. Which brand has the most transactions among users who were created within the past 6 months?

In [37]:
query = """
select b.brandName, count( r.receiptId) as totaltransactions
from receipts r
inner join receiptItems ri on r.receiptId = ri.receiptId
inner join brands b on case when ri.brandCode is not null then ri.brandCode else ri.barcode end = b.brandCode
inner join users u on r.userId = u.userId
where date(u.createdDate) between 
      (select date(max(createdDate), '-6 months') from users) 
      and 
      (select date(max(createdDate)) from users)
group by  b.brandName
order by count( r.receiptId) desc
limit 1
"""
mosttransBrand_perf = pd.read_sql(query, conn)
mosttransBrand_perf

,brandName,totaltransactions
0,PEPSI,74


### **My Findings:**
1. Brand - PEPSI  has most transactions amoung users who created userid within past 6 months

# 3: Evaluate Data Quality Issues in the Data Provided

## Issue 1: Duplicate data in users

In [38]:
### Again loading the file to demonstrate the issue
with open(file_path_users, "r", encoding="utf-8") as file:
            data_users = [json.loads(line) for line in file]
users_df = pd.json_normalize(data_users)
users_df.columns = users_df.columns.str.replace(r'[\._$]', '', regex=True)
users_df = users_df.rename(columns = {'idoid': 'userId', 'createdDatedate': 'createdDate', 
                                      'lastLogindate': 'lastLogin'})
print("total rows before removing duplicates:",users_df['userId'].count())
print("unique users before removing duplicates:",users_df['userId'].nunique())

users_df = users_df.drop_duplicates()
print("total rows after removing duplicates:",users_df['userId'].count())
print("unique users after removing duplicates:",users_df['userId'].nunique())

total rows before removing duplicates: 495
unique users before removing duplicates: 212
total rows after removing duplicates: 212
unique users after removing duplicates: 212


### **My Findings:**
**Issue**: 282 duplicated userids found in users table 

**Impact**: Duplicate data in production databases can lead to serious performance issues, increased storage costs, and data integrity concerns. 

## Issue 2: Some userIds are missing in users table

In [39]:
### Again loading the file to demonstrate the issue
with open(file_path_receipts, "r", encoding="utf-8") as file:
            data_receipts = [json.loads(line) for line in file]
receipts_df = pd.json_normalize(data_receipts)
receipts_df.columns = receipts_df.columns.str.replace(r'[\._$]', '', regex = True)
receipts_df = receipts_df.rename(columns = {'idoid': 'receiptId', 'createDatedate': 'createDate', 
                                            'dateScanneddate': 'dateScanned', 'finishedDatedate': 'finishedDate',
                                            'modifyDatedate': 'modifyDate', 'purchaseDatedate':'purchaseDate',
                                            'pointsAwardedDatedate':'pointsAwardedDate'})

print("unique users in users table: ",users_df['userId'].nunique())
print("unique users in receipts table: ",receipts_df['userId'].nunique())
print("count of users missing from users table: ",receipts_df[~receipts_df['userId'].isin(users_df['userId'] \
                                                                                    .unique())]['userId'].nunique())

unique users in users table:  212
unique users in receipts table:  258
count of users missing from users table:  117


### **My Findings:**
**Issue**: 117 users from receipts data are missing from users database

**Impact**: This discrepancy can lead to data integrity issues, causing inaccurate reporting and inconsistencies in user-based analysis.

## Issue 3: Redundant data in brands table - Category and CategoryCode

In [40]:
with open(file_path_brands, "r", encoding="utf-8") as file:
            data_brands = [json.loads(line) for line in file]
brands_df = pd.json_normalize(data_brands)
brands_df.columns = brands_df.columns.str.replace(r'[\._$]', '', regex = True)
brands_df = brands_df.rename(columns = {'idoid': 'brandId', 'cpgidoid': 'cpgOid', 'cpgref': 'cpgRef', 'name' : 'brandName'})

print("category code is null but category has data",brands_df[~(brands_df['category'].isnull()) & (brands_df['categoryCode'].isnull())]['brandId'].count())
print("category is null but category code has data",brands_df[(brands_df['category'].isnull()) & ~(brands_df['categoryCode'].isnull())]['brandId'].count())
brands_df[['category','categoryCode']].value_counts()

category code is null but category has data 495
category is null but category code has data 0


category                     categoryCode                 
Baking                       BAKING                           359
Candy & Sweets               CANDY_AND_SWEETS                  71
Beer Wine Spirits            BEER_WINE_SPIRITS                 31
Health & Wellness            HEALTHY_AND_WELLNESS              14
Grocery                      GROCERY                           11
Baby                         BABY                               7
Cleaning & Home Improvement  CLEANING_AND_HOME_IMPROVEMENT      6
Bread & Bakery               BREAD_AND_BAKERY                   5
Dairy & Refrigerated         DAIRY_AND_REFRIGERATED             5
Personal Care                PERSONAL_CARE                      4
Beverages                    BEVERAGES                          1
Frozen                       FROZEN                             1
Magazines                    MAGAZINES                          1
Outdoor                      OUTDOOR                            1
Name: count, dtyp

### **My Findings:**
**Issue**: category and categoryCode columns have same data in brands

**Impact**: This redundancy increases storage usage unnecessarily and can cause confusion when querying or analyzing data. It may also lead to inconsistent updates if one column is modified while the other is not

## Issue 3: Special characters and Redundant data in brands table - brandName and brandCode

In [41]:
brands_df[['brandName','brandCode']].value_counts()

brandName                   brandCode                    
Huggies                     HUGGIES                          2
1915 Bolthouse Farms        1915 BOLTHOUSE FARMS             1
test brand @1598290603618   TEST BRANDCODE @1598290603618    1
test brand @1598026274609   TEST BRANDCODE @1598026274609    1
test brand @1598026274643   TEST BRANDCODE @1598026274643    1
                                                            ..
Lunchables                  LUNCHABLES                       1
LÄRABAR™                    LARABAR                          1
L’Oreal Paris - Cosmetics   L’OREAL PARIS - COSMETICS        1
L’Oreal Paris - Hair Color  L’OREAL PARIS - HAIR COLOR       1
test brand @1613158231643   TEST BRANDCODE @1613158231644    1
Name: count, Length: 932, dtype: int64

In [42]:
brands_df['brandName'] = brands_df['brandName'].str.upper()
brands_df['brandCode'] = brands_df['brandCode'].str.upper()
brands_df['brandCode'] = brands_df['brandCode'].str.replace('CODE', '', regex=False).str.strip()
brands_df['brandName'] = brands_df['brandName'].str.replace(r"[^\w\s]", "", regex=True)
brands_df['brandCode'] = brands_df['brandCode'].str.replace(r"[^\w\s]", "", regex=True)
brands_df[
    (brands_df['brandName'] != brands_df['brandCode']) &
    ~(brands_df['brandCode'].isnull()) & 
    (brands_df['brandCode'] != "")
][['brandName', 'brandCode']].value_counts()


brandName                   brandCode         
7 UP                        BRAND                 1
PEOPLE MAGAZINE             511111105114          1
PERONI ITALY                PERONI                1
PILLSBURY MAGAZINE          511111005421          1
POP WORKS  COMPANY POPCORN  POP WORKS  COMPANY    1
                                                 ..
HEALTH MAGAZINE             511111605058          1
                            HEALTH                1
HENRYS HARD SODA            HENRY WEINHARDS       1
HERMAN JOSEPHS              HERMAN                1
YUBAN COFFEE                YUBAN                 1
Name: count, Length: 214, dtype: int64

### **My Findings:**
**Issue**: brandName and brandCode columns have same data in brands. Exception- Out of 932 brandNames, 214 still have different data but some of them are almost same like YUBAN COFFEE   and YUBAN. I believe its data discrepancies and both column have same data. As you can see brandName and brandCode has same data but in different format like brandCode is in capital letters,  brandName has special characters like ™  and brandName does not have word CODE in it

**Impact**: Storing the same information in two columns increases storage costs and affects database efficiency. Differences in formatting (capitalization, special characters, abbreviations) can lead to inconsistent data retrieval and incorrect results in joins or aggregations.

## Issue 4: 8 cpgIds are tagged with multiple cpgRef

In [43]:
brands_df.groupby('cpgOid')['cpgRef'].nunique() \
    .rename('cnt') \
    .reset_index() \
    .groupby('cnt')['cpgOid'] \
    .count()

cnt
1    188
2      8
Name: cpgOid, dtype: int64

### **My Findings:**
**Issue**: 8 cpgIds have multiple cpgRef which are data discrepancies.

**Impact**:  A cpgId should ideally have a single reference (cpgRef). Multiple references may indicate incorrect mappings or duplicate relationships, leading to inconsistent data interpretation

## Issue 5: 2 columns- brandcode and barcode from receipts table are stored in 1 column brandCode in brands table

In [44]:
###### check how brandCode from barnds_df is related with brandCode in final_rewards_df table
print("no of rows with brandcode and brandcode match",final_item_receipts[(final_item_receipts['brandCode'].isin(brands['brandCode']))]['receiptId'].count())
print("no of rows with barcode and brandcode match",final_item_receipts[(final_item_receipts['barcode'].isin(brands['brandCode']))]['receiptId'].count())
print("no of rows with (brandcode or barcode) and brandcode match",final_item_receipts[(final_item_receipts['brandCode'].isin(brands['brandCode'])) | \
                 (final_item_receipts['barcode'].isin(brands['brandCode']))]['receiptId'].count())

no of rows with brandcode and brandcode match 4947
no of rows with barcode and brandcode match 3851
no of rows with (brandcode or barcode) and brandcode match 5788


### **My Findings:**
**Issue**: 2 columns- brandcode and barcode from receipts table are stored in 1 column brandCode in brands table

**Impact**:  Since two different identifiers (brandCode and barcode) are stored in the same field, it can become unclear whether a given brandCode in brands represents a brand-specific code or a barcode. This may lead to incorrect brand mappings

## Issue 6: Null values 

In [45]:
def null_percentage(df, df_name):
    print(f"Null values percentage in {df_name}:")
    print((df.isnull().sum() / len(df) * 100).round(2).astype(str) + '%')
    print("***************************")

null_percentage(users, "users_df")
null_percentage(brands, "brands_df")
null_percentage(receipts, "receipts_df")
null_percentage(final_item_receipts, "final_item_receipts")


Null values percentage in users_df:
active            0.0%
role              0.0%
signUpSource     2.36%
state            2.83%
userId            0.0%
createdDate       0.0%
lastLogin       18.87%
dtype: object
***************************
Null values percentage in brands_df:
brandId        0.0%
cpgOid         0.0%
brandName      0.0%
brandCode    20.05%
topBrand     52.44%
category     13.28%
barcode        0.0%
dtype: object
***************************
Null values percentage in receipts_df:
bonusPointsEarned          51.39%
bonusPointsEarnedReason    51.39%
pointsEarned               45.58%
purchasedItemCount         43.25%
rewardsReceiptStatus         0.0%
totalSpent                 38.87%
userId                       0.0%
receiptId                    0.0%
createDate                   0.0%
dateScanned                  0.0%
finishedDate               49.24%
modifyDate                   0.0%
pointsAwardedDate          52.01%
purchaseDate               40.04%
dtype: object
*************

### **My Findings:**
**Issue**: Many columns in final_item_receipts does not have data, almots more than 90% of the rows does not have data. Similarly, receipts has some columns which does not have almost 50% of data. 

**Impact**:  Large volumes of null values consume unnecessary storage space and increase database size without adding value. Queries that involve these sparse columns may slow down performance, especially if they require filtering, joins, or aggregations on these columns.